In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [2]:
# Supposons que df_players est votre DataFrame préparé
# Il doit contenir les colonnes : 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent'

# Exemple initialisation ou import réel:
# df_players = pd.read_csv('votre_fichier.csv')
# df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(float)

budget = 1039  # Exemple budget total (en 0.1M £)
alpha = -0.05    # Pondération ownership (à ajuster: plus grand = plus pénalisant)

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Créer problème d'optimisation
prob = LpProblem("FPL_Team_Optimizer", LpMaximize)

# Variables binaires pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : maximiser points_per_cost tout en minimisant selected_by_percent
prob += lpSum([
    df_players.loc[i, 'points_per_cost'] * player_vars[i] - 
    alpha * df_players.loc[i, 'selected_by_percent'] * player_vars[i]
    for i in range(len(df_players))
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte : max 3 joueurs par équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i]
        for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Contraintes budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= budget

# Contrainte minimum minutes (exemple)
prob += lpSum([df_players.loc[i, 'minutes'] * player_vars[i] for i in range(len(df_players))]) >= 1000

# Résoudre
prob.solve()

# Extraire liste des joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

# Afficher résultats clés
print(selected_df[['web_name', 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent']])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/f9f8226d06e246d0af8d75a4b9871397-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/f9f8226d06e246d0af8d75a4b9871397-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 5489 RHS
At line 5517 BOUNDS
At line 6258 ENDATA
Problem MODEL has 27 rows, 740 columns and 3362 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 30.177 - 0.00 seconds
Cgl0004I processed model has 27 rows, 688 columns (688 integer (653 of which binary)) and 3154 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.235294
Cbc0038I Solution found of -30.1134
Cbc0038I Cleaned solution of -30.1134
Cbc0038I Before mini branch and bound, 686 integers at bound fixed and 0 continuous


In [3]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,ict_index,selected_by_percent,selected_rank,value_form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90
0,Marc,Guéhi,CRY,Crystal Palace,DEF,47,29.0,0.617021,14.9,21.5,19,1.6,360.0,1278510,240180,261248,14035,0.05,0.00,0.01,0.06,1.00,0.25,6.50
5,Micky,Van de Ven,TOT,Spurs,DEF,46,27.0,0.586957,14.6,30.6,8,1.5,349.0,952781,249627,458715,11350,0.18,0.00,0.01,0.19,1.21,0.26,6.19
44,Pedro,Pedro Porro,TOT,Spurs,DEF,56,21.0,0.375000,20.7,25.3,13,0.9,340.0,852680,62097,499340,56187,0.03,0.00,0.21,0.24,1.23,0.26,7.15
46,Virgil,Virgil,LIV,Liverpool,DEF,61,22.0,0.360656,19.2,28.5,10,1.0,360.0,833042,96776,473958,30712,0.04,0.00,0.02,0.06,0.81,1.00,12.50
60,Marc,Cucurella,CHE,Chelsea,DEF,62,20.0,0.322581,19.8,28.1,11,0.6,293.0,1290518,44116,462854,118583,0.05,0.00,0.14,0.19,0.87,0.61,7.06
25,João Pedro,João Pedro,CHE,Chelsea,FWD,77,33.0,0.428571,33.2,66.7,1,1.3,330.0,2120245,176491,682920,45238,0.41,0.00,0.05,0.46,0.83,0.55,5.18
87,Hugo,Ekitiké,LIV,Liverpool,FWD,88,24.0,0.272727,21.7,29.8,9,0.5,299.0,2103436,72285,778905,179673,0.44,0.00,0.04,0.48,0.84,0.60,3.61
118,Viktor,Gyökeres,ARS,Arsenal,FWD,90,20.0,0.222222,25.2,25.6,12,0.7,306.0,991504,131961,935716,75981,0.62,0.00,0.08,0.70,0.44,0.29,3.53
24,David,Raya,ARS,Arsenal,GKP,55,24.0,0.436364,10.5,23.4,14,0.9,360.0,764685,51566,386494,51012,0.00,2.75,0.00,0.00,0.60,0.25,0.00
59,Martin,Dúbravka,BUR,Burnley,GKP,40,13.0,0.325000,9.3,34.0,5,0.9,360.0,511127,46814,269991,23571,0.00,3.00,0.00,0.00,2.29,1.75,0.00


In [4]:
selected_df['price'].sum()

np.int64(1033)

In [5]:
selected_df['selected_by_percent'].sum()

np.float64(491.80000000000007)